# Pairs Traiding through Unsupervised Learning

In [ ]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import os

tqdm.pandas()

MIN_YEAR=2018 # 1980
MAX_YEAR=2021
CHUNKS = 10000

FILEPATH = f"./data/historic_characteristics.csv"
FILEPATH_PARQ = f"./data/historic_characteristics_{MIN_YEAR}_{MAX_YEAR}.parquet"
FILEPATH_MOM_PARQ = f"./data/data_mom_{MIN_YEAR}_{MAX_YEAR}.parquet"
FILEPATH_CLEAN_PARQ = f"./data/data_cleaning_{MIN_YEAR}_{MAX_YEAR}.parquet"
FILEPATH_PRE_PARQ = f"./data/data_preprocessed_{MIN_YEAR}_{MAX_YEAR}.parquet"

CLUSTERS_FILEPATH = f"./data/clusters_data_{MIN_YEAR}_{MAX_YEAR}.pkl"
CLUSTERS_MEM_FILEPATH = f"./data/clusters_mem_data_{MIN_YEAR}_{MAX_YEAR}.pkl"
SIGNALS_FILEPATH = f"./data/signals_{MIN_YEAR}_{MAX_YEAR}.pkl"
PCA_FILEPATH = f"./data/pca_{MIN_YEAR}_{MAX_YEAR}.pkl"

FEATURES = [
    "DATE", "absacc", "acc", "aeavol", "age", "agr", "baspread", "beta", "betasq", "bm",
    "bm_ia", "cash", "cashdebt", "cashpr", "cfp", "cfp_ia", "chatoia", "chcsho", "chempia",
    "chinv", "chmom", "chpmia", "chtx", "cinvest", "convind", "currat", "depr", "divi",
    "divo", "dolvol", "dy", "ear", "egr", "ep", "gma", "herf", "hire", "idiovol", "ill",
    "indmom", "invest", "lev", "lgr", "maxret", "mom1m", "ms", "mve_ia", "mvel1", "nincr",
    "operprof", "pchcapx_ia", "pchcurrat", "pchdepr", "pchgm_pchsale", "pchquick",
    "pchsale_pchrect", "pctacc", "permno", "pricedelay", "ps", "quick", "rd", "retvol",
    "roaq", "roeq", "roic", "rsup", "salecash", "salerec", "securedind", "sgr", "sic2",
    "sin", "sp", "std_dolvol", "std_turn", "tang", "tb", "turn", "zerotrade"
]
WINDOW = 48
MOM_FEATURES = [f"mom{i}m" for i in range(1, WINDOW + 1)]

# Data Wrangling

## Parquet Dataset Creation

The dataset is large, around 3GB of company characteristics from 1985 to 2021. This dataset has been currated for the papers ["Empirical Asset Pricing via Machine Learning"](https://dachxiu.chicagobooth.edu/download/ML.pdf)(2018) and ["Autoencoder Asset Pricing Models." ](https://www.sciencedirect.com/science/article/abs/pii/S0304407620301998)(2019) by Shihao Gu, Bryan Kelly and Dacheng Xiu. The raw format is available for download from the authors [personal website](https://sites.google.com/view/agoyal145) (or reach out to me for a currated dataset). The dataset has 94 1 month Lagged Firm Characteristics (as the CRSP releases these with a month delay, from the notes in their papers). Note that this CRSP datasets don't have tickers or company names, but use a permanent indentifier instead, which if you have a bloomberg terminal or access to a research site that brokers this data, you can easily convert to the company ticker.

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

if not os.path.exists(FILEPATH_PARQ):
    chars_df = pd.read_csv(FILEPATH)[FEATURES]
    chars_df['DATE'] = pd.to_datetime(chars_df['DATE'], format='%Y%m%d')
    chars_df = chars_df[(chars_df['DATE'].dt.year >= MIN_YEAR) & (chars_df['DATE'].dt.year <= MAX_YEAR)]
    chars_df = chars_df.sort_values("DATE")
    chars_df.to_parquet(FILEPATH_PARQ, index=False, compression="snappy")
else:
    chars_df = pd.read_parquet(FILEPATH_PARQ)

chars_df.head(5)

## Sanitinzation and Feature Engineering

To sanitinize we drop any company with insufficiant data to fill a window, and we fill any missing characteristic with the median of that window. We then perform a rolling window to calculate the MOM factor for 2 to 64 months, the data already has a rolling 1 month momentum. To read about momentum stratgies check out the article [Momentum and Reversion Trading Signals Analysis](https://medium.com/call-for-atlas/momentum-and-reversion-the-poor-mans-trading-strategies-9b8e1e6d3496).

Imputting the MOM features for 49 months:

For $i = 1, mom_i = r_{t-1}, \quad i = 1$ where $r_{t-1}$ denotes the return in the previous month.

For $i > 1$, we calculate the momentum over a window of $i$ months is given by:
$$
mom_i = \left( \prod_{j=t-i}^{t-2} (r_j + 1) \right) - 1, \quad i \in \{2, \ldots, 48\},
$$
where \(r_j\) denotes the return in month \(j\).



In [ ]:
def interpolate_with_median(group):
    rolling_median = group.rolling(window=WINDOW, min_periods=1).median()
    group= group.fillna(rolling_median).bfill()
    return group

if not os.path.exists(FILEPATH_PRE_PARQ):
    valid_groups = chars_df.groupby('permno').filter(lambda x: len(x) >= WINDOW and x[MOM_FEATURES[0]].isna().sum() <= 2)
    for i in tqdm(range(2, WINDOW + 1), desc="moms"):
        rolling_func = lambda x: (x + 1).rolling(window=i).apply(np.prod, raw=True) - 1
        valid_groups[f'mom{i}m'] = valid_groups.groupby('permno')[MOM_FEATURES[0]].transform(rolling_func)

    numerical_columns = valid_groups.select_dtypes(include=['float64', 'int64']).columns

    tqdm.pandas(desc="interpolate_with_median")
    valid_groups[numerical_columns]= valid_groups.groupby('permno')[numerical_columns].progress_transform(lambda x: interpolate_with_median(x))
    valid_groups.to_parquet(FILEPATH_PRE_PARQ, index=False, compression="snappy")
    chars_df = valid_groups
else:
    chars_df = pd.read_parquet(FILEPATH_PRE_PARQ)

chars_df.tail(5)

## Firm Characteristics

| Acronym  | Firm characteristic                                           | Acronym    | Firm characteristic                                       |
|----------|--------------------------------------------------------------|------------|----------------------------------------------------------|
| absacc   | Absolute accruals                                            | invest     | Capital expenditures and inventory                        |
| acc      | Working capital accruals                                     | IPO        | New equity issue                                          |
| aeavol   | Abnormal earnings announcement volume                        | lev        | Leverage                                                  |
| age      | # years since first Compustat coverage                       | lgr        | Growth in long-term debt                                  |
| agr      | Asset growth                                                  | maxret     | Maximum daily return                                      |
| baspread | Bid-ask spread                                               | ms         | Financial statement score                                 |
| beta     | Beta                                                         | mve        | Size                                                      |
| betasq   | Beta squared                                                 | mve ia     | Industry-adjusted size                                    |
| bm       | Book-to-market                                               | nincr      | Number of earnings increases                              |
| bm ia    | Industry-adjusted book to market                             | operprof   | Operating profitability                                   |
| cash     | Cash holdings                                                | pchcapx ia | Industry adjusted % change in capital expenditures        |
| cashdebt | Cash flow to debt                                            | pchcurrat  | % change in current ratio                                 |
| cashpr   | Cash productivity                                            | pchdepr    | % change in depreciation                                  |
| cfp      | Cash flow to price ratio                                     | pchgm      | % change in gross margin                                  |
| cfp ia   | Industry-adjusted cash flow to price ratio                   | pchsale    | % change in sales                                         |
| chatoia  | Industry-adjusted change in asset turnover                   | pchquick   | % change in quick ratio                                   |
| chcsho   | Change in shares outstanding                                 | pctacc     | Percent accruals                                          |
| chempia  | Industry-adjusted change in employees                        | pricedelay | Price delay                                               |
| chinv    | Change in inventory                                          | ps         | Financial statements score                                |
| chmom    | Change in 6-month momentum                                   | quick      | Quick ratio                                               |
| chpmia   | Industry-adjusted change in profit margin                    | rd         | R&D increase                                              |
| chtx     | Change in tax expense                                        | retvol     | Return volatility                                         |
| cinvest  | Corporate investment                                         | roaq       | Return on assets                                          |
| convind  | Convertible debt indicator                                   | roeq       | Return on equity                                          |
| currat   | Current ratio                                                | roic       | Return on invested capital                                |
| depr     | Depreciation / PP&E                                          | rsup       | Revenue surprise                                          |
| divi     | Dividend initiation                                          | sgr        | Sales growth                                              |
| divo     | Dividend omission                                            | sin        | Sin stocks                                                |
| dolvol   | Dollar trading volume                                        | SP         | Sales to price                                            |
| dy       | Dividend to price                                            | std dolvol | Volatility of liquidity (dollar trading volume)          |
| ear      | Earnings announcement return                                 | std turn   | Volatility of liquidity (share turnover)                  |
| egr      | Growth in common shareholder equity                          | sue        | Unexpected quarterly earnings                             |
| ep       | Earnings to price                                            | tang       | Debt capacity/firm tangibility                            |
| gma      | Gross profitability                                          | tb         | Tax income to book income                                 |
| herf     | Industry sales concentration                                 | turn       | Share turnover                                            |
| hire     | Employee growth rate                                         | zerotrade  | Zero trading days                                         |
| idiovol  | Idiosyncratic return volatility                              |            |                                                           |
| ill      | Illiquidity                                                  |            |                                                           |
| indmom   | Industry momentum                                            |            |                                                           |


Additionally, there are PERMNO columns to ID the company, and a **SIC code** to ID the industry from [NAICS](https://www.naics.com/sic-codes-industry-drilldown/) to compliment the industires momentum **INDMOM**.


For MOM - this is the equities' acceleration measure and is calculated as follows:

$$
\text{MOM}_{\text{1 month}} = \left( \frac{\text{Price at End of Month} - \text{Price at Start of Month}}{\text{Price at Start of Month}} \right) \times 100
$$

Although we don't have a method to convert PERMNO to the actual stock and calculate its price, MOM already provides that change for 1 month.

## Dim Reduction with PCA

We perform standardization and PCA at 95% variance, to center the data's means for the clustering algorithims and reduce its dimensionality.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

MAX_VARIANCE = 0.99

chars_pca_df = chars_df.copy()
if os.path.exists(PCA_FILEPATH):
    pca_result_df = pd.read_pickle(PCA_FILEPATH)
else:
    scaler = StandardScaler()
    pca = PCA(MAX_VARIANCE)
    features_df = chars_pca_df.drop(['DATE', 'permno'], axis=1).bfill()
    pipeline = Pipeline([('scaler', scaler), ('pca', pca)])
    pca_df = pipeline.fit_transform(features_df)

    pca_result_df = pd.DataFrame(data=pca_df, index=chars_pca_df.index)
    pca_components_cols = pca_result_df.columns
    pca_result_df['permno'] = chars_pca_df['permno']
    pca_result_df[MOM_FEATURES] = chars_pca_df[MOM_FEATURES]
    pca_result_df['DATE'] = chars_pca_df['DATE']

    pca_result_df.to_pickle(PCA_FILEPATH)

pca_result_df.tail(5)

# Cluster Agglomerative

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import AgglomerativeClustering

def train_clusters(pca_result_df):
    models_dfs = []

    cluster_membership = []
    for month, data in tqdm(pca_result_df.groupby("DATE"), desc="Processing months"):
        pca_data = data[pca_components_cols]
        if len(pca_data) < 2:
            print(f"Skipping {month} due to insufficient data.")
            continue

        neigh = NearestNeighbors(n_neighbors=2)
        nbrs = neigh.fit(pca_data)
        distances, _ = nbrs.kneighbors(pca_data)

        distances = np.sort(distances, axis=0)
        distances = distances[:, 1]
        distance_alpha_thresh = np.percentile(distances, 30)
        agg_model = AgglomerativeClustering(n_clusters=None, distance_threshold=distance_alpha_thresh, linkage='average')
        agg_model.fit(pca_data)

        cluster_df = pd.DataFrame(data['permno'].copy(), index=data.index)
        cluster_df['Cluster'] = agg_model.labels_
        cluster_df['DATE'] = month
        cluster_df[MOM_FEATURES[0]] = data[MOM_FEATURES[0]]

        cluster_membership.append(cluster_df)

        models_dfs.append({'DATE': month, 'NumClusters': agg_model.n_clusters_})

    models_df = pd.DataFrame(models_dfs)
    cluster_membership_df = pd.concat(cluster_membership, ignore_index=False)

    return models_df, cluster_membership_df

if os.path.exists(CLUSTERS_FILEPATH) and os.path.exists(CLUSTERS_MEM_FILEPATH) :
    models_df = pd.read_pickle(CLUSTERS_FILEPATH)
    cluster_membership_df = pd.read_pickle(CLUSTERS_MEM_FILEPATH)
else:
    models_df, cluster_membership_df = train_clusters(pca_result_df)
    models_df.to_pickle(CLUSTERS_FILEPATH)
    cluster_membership_df.to_pickle(CLUSTERS_MEM_FILEPATH)

pca_result_df['Cluster'] = cluster_membership_df['Cluster']

In [ ]:
models_df.tail(5)

# Trade Simulation

The trade will take the following steps:
1. Check that the security is in the cluster.
2. Get cross-sectional standard dev.
3. split into deciles.
4. if first > last by > 1 std - there is a statarb opportunity.
5. Select Long-Short for that month, and close securities from previous month which have reversed back to their normal distance.

In [ ]:
def statarb_signals(group):
    rets = group[MOM_FEATURES[0]].mean()
    std_dev = group[MOM_FEATURES[0]].std()
    group['Decile'] = pd.qcut(group[MOM_FEATURES[0]], 10, labels=False, duplicates='drop') + 1

    over_cond = (group['Decile'] == 10) & (group[MOM_FEATURES[0]] > rets + std_dev)
    under_cond = (group['Decile'] == 1) & (group[MOM_FEATURES[0]] < rets - std_dev)
    overs = set(group[over_cond]['permno'].tolist())
    unders = set(group[under_cond]['permno'].tolist())

    trade_ops = []
    if overs or unders:
        trade_ops.append({
            'DATE': group['DATE_TRADE'].unique()[0],
            'Cluster': group['Cluster'].tolist(),
            'overs': overs,
            'unders': unders
            })

    return trade_ops

pca_result_df['DATE_TRADE'] = pca_result_df.groupby('permno')['DATE'].shift(-1).ffill()
if os.path.exists(SIGNALS_FILEPATH):
    trade_opportunities_df = pd.read_pickle(SIGNALS_FILEPATH)
else:
    trade_opportunities = pca_result_df.groupby(['DATE_TRADE', 'Cluster']).progress_apply(statarb_signals)
    trade_opportunities = [item for sublist in trade_opportunities for item in sublist]
    trade_opportunities_df = pd.DataFrame(trade_opportunities)
    trade_opportunities_df.to_pickle(SIGNALS_FILEPATH)

trade_opportunities_df.tail(5)

## PnL

In [ ]:
filtered_trade_opportunities_df = trade_opportunities_df[
    (trade_opportunities_df['overs'].apply(len) > 0) & (trade_opportunities_df['unders'].apply(len) > 0)
]
filtered_trade_opportunities_df.tail(5)

In [ ]:
TRADE_FEATURE = ["DATE"]

asset_df = pca_result_df.reset_index()[['DATE', 'permno', MOM_FEATURES[0]]].copy()

monthly_profit_loss = []

trade_opportunities_df['prev_overs'] = trade_opportunities_df['overs'].shift(1).bfill()
trade_opportunities_df['prev_unders'] = trade_opportunities_df['unders'].shift(1).bfill()
trade_opportunities_df['overs_delta'] = trade_opportunities_df.apply(lambda row: row['prev_overs'].difference(row['overs']), axis=1)
trade_opportunities_df['unders_delta'] = trade_opportunities_df.apply(lambda row: row['prev_unders'].difference(row['unders']), axis=1)

overs_df = trade_opportunities_df[TRADE_FEATURE +  ['overs_delta']].explode('overs_delta').rename(columns={'overs_delta': 'permno'})
unders_df = trade_opportunities_df[TRADE_FEATURE +  ['unders_delta']].explode('unders_delta').rename(columns={'unders_delta': 'permno'})

overs_df = pd.merge(overs_df, asset_df, left_on=['DATE', 'permno'], right_on=['DATE', 'permno'])
unders_df = pd.merge(unders_df, asset_df, left_on=['DATE', 'permno'], right_on=['DATE', 'permno'])

unders_df.tail(5)

In [ ]:
overs_df[overs_df["permno"] == 79851]

In [ ]:
unders_df[unders_df["permno"] == 79851]

## Performance

Equal Weighted Returns:

$$
R_p = \sum_{i=1}^{n} (w_i \cdot r_i)
$$

In [ ]:
overs_df = overs_df.groupby('DATE')[MOM_FEATURES[0]].mean()
unders_df = unders_df.groupby('DATE')[MOM_FEATURES[0]].mean()

overs_df.tail(5)

In [ ]:
unders_df.tail(5)

In [ ]:
over_ret = overs_df.mul(-1).add(1).cumprod().sub(1)
under_ret = unders_df.add(1).cumprod().sub(1)

over_ret.tail(5)

In [ ]:
under_ret.tail(5)